In [2]:
!pip install gradio scikit-learn pandas numpy

import numpy as np
import pandas as pd
import gradio as gr
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# ---------------------------
# 1) Train model on synthetic logins
# ---------------------------
def generate_synthetic_logins(n_samples=2000, anomaly_ratio=0.1, random_state=42):
    rng = np.random.RandomState(random_state)

    n_anomalies = int(n_samples * anomaly_ratio)
    n_normals   = n_samples - n_anomalies

    normal_location_change = rng.binomial(1, 0.1, n_normals)
    normal_new_device      = rng.binomial(1, 0.1, n_normals)
    normal_ip_rep          = rng.choice([0, 1], size=n_normals, p=[0.85, 0.15])
    normal_failed_attempts = np.clip(rng.poisson(0.5, n_normals), 0, 5)
    normal_hour            = np.clip(rng.normal(16, 3, n_normals), 0, 23)

    anom_location_change = rng.binomial(1, 0.8, n_anomalies)
    anom_new_device      = rng.binomial(1, 0.7, n_anomalies)
    anom_ip_rep          = rng.choice([1, 2], size=n_anomalies, p=[0.3, 0.7])
    anom_failed_attempts = np.clip(rng.poisson(3.5, n_anomalies), 0, 5)
    anom_hour            = np.clip(rng.normal(3, 3, n_anomalies), 0, 23)

    df = pd.DataFrame({
        "location_change": np.concatenate([normal_location_change, anom_location_change]),
        "new_device":      np.concatenate([normal_new_device,      anom_new_device]),
        "ip_reputation":   np.concatenate([normal_ip_rep,          anom_ip_rep]),
        "failed_attempts": np.concatenate([normal_failed_attempts, anom_failed_attempts]),
        "hour":            np.concatenate([normal_hour,            anom_hour])
    })
    return df

df = generate_synthetic_logins()
features = ["location_change","new_device","ip_reputation","failed_attempts","hour"]

scaler = StandardScaler()
X = scaler.fit_transform(df[features].values)

model = IsolationForest(n_estimators=200, contamination=0.1, random_state=42)
model.fit(X)

train_scores = -model.score_samples(X)
q_low  = np.quantile(train_scores, 0.1)
q_high = np.quantile(train_scores, 0.9)

# ---------------------------
# 2) Evaluation function (with automatic Nafath step)
# ---------------------------
def evaluate_login(national_id, password, location, device, ip, fails, hour):
    if not national_id or not password:
        return "❗ Please enter National ID and password."

    # encode inputs
    loc_val = 1 if location == "Different region" else 0
    dev_val = 1 if device == "New device" else 0
    ip_val  = {"Good":0,"Suspicious":1,"Bad":2}[ip]
    fails   = int(fails)
    hour    = float(hour)

    x = np.array([[loc_val,dev_val,ip_val,fails,hour]])
    x_scaled = scaler.transform(x)

    # 1) ML risk
    raw = -model.score_samples(x_scaled)[0]
    normalized_ml = np.clip((raw - q_low)/(q_high - q_low + 1e-8), 0, 1)
    risk_ml = normalized_ml * 100

    # 2) Rule-based risk
    risk_rules = 0
    if loc_val == 1: risk_rules += 25
    if dev_val == 1: risk_rules += 20
    if ip_val == 1:  risk_rules += 20
    if ip_val == 2:  risk_rules += 35
    if fails >= 1:   risk_rules += 10
    if fails >= 3:   risk_rules += 10
    if hour < 5 or hour > 23: risk_rules += 15
    risk_rules = max(0.0, min(100.0, risk_rules))

    # 3) Combine
    final = 0.5 * risk_ml + 0.5 * risk_rules

    # safety rule for very normal behavior
    if (
        loc_val == 0 and
        dev_val == 0 and
        ip_val  == 0 and
        fails   <= 1 and
        6 <= hour <= 23
    ):
        final = min(final, 25.0)

    final = round(final, 2)

    # 4) Decision logic + Nafath step
    if final < 35:
        level = "SAFE"
        decision = "ALLOW"
        return (
            f"User: {national_id}\n"
            f"Risk: {final}\n"
            f"Level: {level}\n"
            f"Decision: {decision}\n"
            f"Access granted directly (no Nafath required)."
        )

    elif final < 70:
        level = "SUSPICIOUS"
        # هنا نحاكي نفاذ أوتوماتيكياً
        nafath_result = "VERIFIED"  # في النظام الحقيقي هذا يجي من API لنفاذ

        if nafath_result == "VERIFIED":
            final_decision = "ALLOW"
            return (
                f"User: {national_id}\n"
                f"Risk: {final}\n"
                f"Level: {level}\n"
                f"Step-up: Nafath biometric verification required.\n"
                f"Nafath result: {nafath_result}.\n"
                f"Final decision: {final_decision}\n"
                f"User is allowed to access after Nafath."
            )
        else:
            final_decision = "BLOCK"
            return (
                f"User: {national_id}\n"
                f"Risk: {final}\n"
                f"Level: {level}\n"
                f"Step-up: Nafath biometric verification required.\n"
                f"Nafath result: FAILED.\n"
                f"Final decision: {final_decision}\n"
                f"Login blocked after failed Nafath verification."
            )

    else:
        level = "HIGH RISK"
        decision = "BLOCK"
        return (
            f"User: {national_id}\n"
            f"Risk: {final}\n"
            f"Level: {level}\n"
            f"Decision: {decision}\n"
            f"Login blocked immediately due to high risk."
        )

# ---------------------------
# 3) Gradio UI (single Login button)
# ---------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🔐 VisionGuard – Smart Login with Automatic Nafath Step-up")
    gr.Markdown("Simulated Absher login with risk engine and Nafath integration.")

    national_id = gr.Textbox(label="National ID / Iqama")
    password    = gr.Textbox(label="Password", type="password")

    gr.Markdown("### Internal risk signals (collected automatically in real system)")
    location = gr.Radio(["Same region","Different region"], label="Login Location")
    device   = gr.Radio(["Known device","New device"], label="Device")
    ip       = gr.Radio(["Good","Suspicious","Bad"], label="IP Reputation")
    fails    = gr.Slider(0,5,step=1,label="Failed Attempts", value=0)
    hour     = gr.Slider(0,23,step=1,label="Login Hour", value=16)

    btn = gr.Button("Login")
    output = gr.Textbox(label="Result", lines=10)

    btn.click(
        evaluate_login,
        inputs=[national_id,password,location,device,ip,fails,hour],
        outputs=output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db2fe7065a6daf9665.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
